In [1]:
import os
import datetime
import pathlib
import pandas as pd
from PIL import Image

import torch
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Subset

import albumentations
# from albumentations.pytorch import ToTensor

from sklearn.model_selection import KFold, StratifiedKFold

from src.train import train
from src.inference import inference
from src.evaluation import evaluate
from src.model import Net
from src.loss import my_loss
from src.dataset import TrainDataset, TestDataset, get_train_dataloader, get_valid_dataloader, get_test_dataloader

In [2]:
# arguments
model_num = 1
MODEL_PATH = os.path.join('./models/', datetime.datetime.now().strftime('%Y-%m-%d') + f' model {model_num}')
submission_file_name = datetime.datetime.now().strftime('%Y-%m-%d') + f' model {model_num}.csv'
pathlib.Path(MODEL_PATH).mkdir(parents=True, exist_ok=True) 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_epochs = 15

batch_size = 32
num_workers = 4
learning_rate = 0.00005
weight_decay = 0.01

In [3]:
# dataset path
train_dir = '/opt/ml/input/data/train'
test_dir = '/opt/ml/input/data/eval'

# csv files
train_csv = pd.read_csv(os.path.join(train_dir, 'train.csv'))
test_csv = pd.read_csv(os.path.join(test_dir, 'info.csv'))

# image directories
train_image_dir = os.path.join(train_dir, 'images')
test_image_dir = os.path.join(test_dir, 'images')

train_image_paths = [os.path.join(train_image_dir, img_id) for img_id in train_csv.path]
test_image_paths = [os.path.join(test_image_dir, img_id) for img_id in test_csv.ImageID]

In [4]:
# Define transformer
train_transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
#     transforms.RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

test_transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

# train_transform = albumentations.Compose([
#     albumentations.Resize(512, 384), 
# #     albumentations.HorizontalFlip(0.5),
#     ToTensor(),
#     albumentations.Normalize(mean=(0.5,), std=(0.2,))
# ])

# test_transform = albumentations.Compose([
#     albumentations.Resize(512, 384),
#     ToTensor(),
#     albumentations.Normalize(mean=(0.5,), std=(0.2,))
# ])


In [5]:
# Get train, test dataset
train_dataset = TrainDataset(train_image_paths, train_csv, train_transform)
test_dataset = TestDataset(test_image_paths, test_transform)

In [6]:
# Divide dataset with KFold and train
kfold = KFold(n_splits=5)
criterion = my_loss()

max_acc = 0

for fold, (train_index, valid_index) in enumerate(kfold.split(train_dataset), 1):    
    train_sub = Subset(train_dataset, train_index)
    valid_sub = Subset(train_dataset, valid_index)
    
    train_loader = get_train_dataloader(train_sub, batch_size, num_workers)
    valid_loader = get_valid_dataloader(valid_sub, batch_size, num_workers)
    
    # Create model
    model = Net()
    model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=learning_rate, weight_decay=weight_decay)
    # lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    lr_scheduler = None
    
    for epoch in range(num_epochs):
        train_correct, train_total = train(model, train_loader, optimizer, criterion, device)
        valid_correct, valid_total = inference(model, valid_loader, device)
        
        train_acc = 100.*train_correct/train_total
        valid_acc = 100.*valid_correct/valid_total
        
        print(f"[Fold {fold}] Epoch #{epoch} train_acc: {train_acc:.2f}, valid_acc: {valid_acc:.2f}")
        
        if max_acc < valid_acc:
            final_path = os.path.join(MODEL_PATH, f'fold_{fold}_epoch_{epoch}_model.pth')
            torch.save(model.state_dict(), final_path)
            print(f'model saved at [fold {fold}] Epoch #{epoch}')
            max_acc = valid_acc

[Fold 1] Epoch #0 train_acc: 85.07, valid_acc: 80.50
model saved at [fold 1] Epoch #0
[Fold 1] Epoch #1 train_acc: 96.12, valid_acc: 80.08
[Fold 1] Epoch #2 train_acc: 97.71, valid_acc: 82.51
model saved at [fold 1] Epoch #2
[Fold 1] Epoch #3 train_acc: 98.01, valid_acc: 80.45
[Fold 1] Epoch #4 train_acc: 97.98, valid_acc: 79.23
[Fold 1] Epoch #5 train_acc: 98.02, valid_acc: 77.17
[Fold 1] Epoch #6 train_acc: 98.36, valid_acc: 82.49
[Fold 1] Epoch #7 train_acc: 98.31, valid_acc: 78.84
[Fold 1] Epoch #8 train_acc: 98.66, valid_acc: 82.75
model saved at [fold 1] Epoch #8
[Fold 1] Epoch #9 train_acc: 98.28, valid_acc: 81.61
[Fold 1] Epoch #10 train_acc: 98.72, valid_acc: 84.23
model saved at [fold 1] Epoch #10
[Fold 1] Epoch #11 train_acc: 98.49, valid_acc: 80.82
[Fold 1] Epoch #12 train_acc: 98.31, valid_acc: 83.54
[Fold 1] Epoch #13 train_acc: 99.35, valid_acc: 83.54
[Fold 1] Epoch #14 train_acc: 98.59, valid_acc: 83.84
[Fold 2] Epoch #0 train_acc: 85.56, valid_acc: 78.36
[Fold 2] Epoch

In [7]:
# Evaluate on test set
# final_path = './models/2021-04-01 model 1/fold_5_epoch_9_model.pth'
model = Net()
model.load_state_dict(torch.load(final_path))
model.to(device)

test_loader = get_test_dataloader(test_dataset)
evaluate(model, test_loader, test_csv, test_dir, submission_file_name, device)

test inference is done!
